### 과제2
아래 가이드에 따라 고객별 연간 구매데이터를 생성하고 아래사항을 수행하기(관측치 1,000개)

#### Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.

거래 번호
- id : 1 ~ 1000번 일련번호 - 개시 후 1천 건의 거래가 있었고 걍 거기에 넘버링한 거라고 이해해보자

고객별 속성 데이터
- cusno : 고객번호 1000개 랜덤
- gender : 0,1 정수 난수 생성
- age : 0 ~ 80사이 정수 난수 생성
- region : 1 ~ 10 사이 정수 난수 생성

거래 데이터
- (day : 요일)
- (date : 2020-01-01 ~ 2021-12-31 사이의 날짜 1000개 생성)
- product_20 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- product_21 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- price_20 : 1000 ~ 50000 사이 실수 난수 생성
- price_21 : 1000 ~ 50000 사이 실수 난수 생성
- quantity_20 : 1 ~ 100 사이 정수 난수 생성
- quantity_21 : 1 ~ 100 사이 정수 난수 생성
- (time_20 : 01 ~ 24 사이 정수 난수 생성)
- (time_21 : 01 ~ 24 사이 정수 난수 생성)
- amount_20 : price_20 * quantity_20
- amount_21 : price_21 * quantity_21
- sales : 20년 대비 21년구매금액이 증가면 1, 감소면 0



※주의할 점
- date는 product20,21에 주의해서 작성
    - day는 date의 요일
    - time_20,21는 date의 시간

In [7]:
import numpy as np
import pandas as pd
import random

np.random.seed(100)

#고객 데이터프레임 생성
cusno=pd.Series(np.random.randint(1000,5000,size=1000))
gender=pd.Series(np.random.randint(0,2,size=1000))
age=pd.Series(np.random.randint(0,80,size=1000))
region=pd.Series(np.random.randint(1,11,size=1000))

customer=pd.concat([id,cusno,gender,age,region],axis=1)
customer.rename(columns={0:'id',1:'cusno',2:'gender',3:'age',4:'region'},
               inplace=True)


#구매 데이터프레임 생성
id=pd.Series(np.arange(1,1001))
product_20=pd.Series(np.random.choice([10,20,30],size=1000))
product_21=pd.Series(np.random.choice([10,20,30],size=1000))
price_20=pd.Series(np.random.uniform(1000,50000,size=1000).round(2))
price_21=pd.Series(np.random.uniform(1000,50000,size=1000).round(2))
quantity_20=pd.Series(np.random.randint(1,100,size=1000) )
quantity_21=pd.Series(np.random.randint(1,100,size=1000))
amount_20=pd.Series(np.array(price_20*quantity_20).round(2))
amount_21=pd.Series(np.array(price_21*quantity_21).round(2))
sales=pd.Series(np.where(amount_20<amount_21,1,0))

purchase=pd.concat([product_20,product_21,price_20,
                    quantity_20,quantity_21,amount_20,
                    amount_21,sales],axis=1)
purchase.rename(columns={0:'product_20',
                         1:'product_21',
                         2:'price_20',
                         3:'price_21',
                         4:'quantity_20',
                         5:'quantity_21',
                         6:'amount_20',
                         7:'amount_21',
                         8:'sales'},
               inplace=True)


#데이터프레임 합치기
df=customer.join(purchase,how='outer')
df.head()

,id,cusno,gender,age,region,product_20,product_21,price_20,price_21,quantity_20,quantity_21,amount_20,amount_21
0,1,2544,1,48,1,20,10,23225.68,96,91,2229665.28,2183282.01,0
1,2,3840,0,20,8,10,20,27716.36,92,5,2549905.12,52445.30,0
2,3,2859,0,54,7,10,10,24624.63,7,92,172372.41,2348875.00,1
3,4,4943,1,79,1,30,20,44141.70,51,20,2251226.70,204012.80,0
4,5,4927,0,10,3,30,20,26712.91,68,38,1816477.88,590478.20,0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1000 non-null   int32  
 1   cusno        1000 non-null   int32  
 2   gender       1000 non-null   int32  
 3   age          1000 non-null   int32  
 4   region       1000 non-null   int32  
 5   product_20   1000 non-null   int32  
 6   product_21   1000 non-null   int32  
 7   price_20     1000 non-null   float64
 8   price_21     1000 non-null   int32  
 9   quantity_20  1000 non-null   int32  
 10  quantity_21  1000 non-null   float64
 11  amount_20    1000 non-null   float64
 12  amount_21    1000 non-null   int32  
dtypes: float64(3), int32(10)
memory usage: 62.6 KB


In [3]:
print(df.product_20.value_counts())
print(df.product_21.value_counts())

10    336
20    332
30    332
Name: product_20, dtype: int64
30    343
10    331
20    326
Name: product_21, dtype: int64


In [32]:
df['cusno'].duplicated().sum()

118

#### Q2.data 유의미하게 5가지 도출

1. 재구매가 자주 이루어진 고객은 보통 여성들
    - cusno의 랜덤 범위를 줄이고, 중복된 cusno의 gender을 거의 여성으로 전환
2. 21년도 이탈 고객은 주로 남성들
    - product_20가 np.nan이 아니고, product_21 데이터 90개를 np.nan화, gender의 80%를 남성으로 전환
3. 코로나로 인해, 21년도에 새로 유입된 소비자는 20년도보다 10%p 상승
    - product_20 데이터 90개를 np.nan화,cusno 200개 중복 x
4. 코로나로 인해, 20년도 대비 21년도 30-40대의 생필품 구매가 증가해 중저가 소비가 증가함
    - 세부분류 코드 생성(10:{1: 세제 등 생필품, 2.식자재, 3.},등등
                       20:{:} 
                       30:{}) 
    - product_21이 10,20이면, quantity_21 증가

##### 1. 20년도, 21년도 재구매가 이루어진 고객수의 80%가 여성

In [33]:
#재구매 횟수
pd.set_option('display.max_row',120)

re_purchase=df['cusno'].duplicated()
print(df.loc[re_purchase,'id'].value_counts())
print(df.loc[re_purchase,'id'].value_counts().unique())


142    1
796    1
871    1
866    1
864    1
853    1
846    1
844    1
839    1
837    1
826    1
820    1
818    1
804    1
788    1
875    1
787    1
783    1
777    1
769    1
767    1
760    1
759    1
755    1
751    1
743    1
741    1
736    1
872    1
877    1
158    1
950    1
987    1
981    1
980    1
975    1
974    1
971    1
969    1
964    1
959    1
953    1
952    1
951    1
949    1
888    1
942    1
936    1
935    1
932    1
929    1
917    1
915    1
914    1
910    1
905    1
901    1
896    1
733    1
725    1
720    1
387    1
541    1
540    1
531    1
523    1
515    1
513    1
485    1
481    1
446    1
440    1
430    1
398    1
341    1
715    1
340    1
324    1
319    1
311    1
301    1
298    1
291    1
286    1
266    1
255    1
243    1
228    1
549    1
555    1
561    1
563    1
707    1
705    1
700    1
699    1
698    1
695    1
691    1
685    1
684    1
680    1
676    1
672    1
661    1
660    1
652    1
641    1
636    1
634    1
632    1
6

In [34]:
#재구매 고객

before_all=len(df[re_purchase])   #재구매 고객수
print('총 재구매 고객 수: ',before_all)

print('재구매 고객의 80%: ',int(before_all*0.8))
pd.set_option('display.max_row',120)

df.loc[re_purchase,'id'].value_counts().unique()
before_f=len(df.loc[(re_purchase)&(df['gender']==1),'gender']) #그 중 여성의 수
before_m=len(df.loc[(re_purchase)&(df['gender']==0),'gender'])
print('재구매 고객 중 여성 수: ',before_f)
print('재구매 고객 중 남성 수: ',before_m)

총 재구매 고객 수:  118
재구매 고객의 80%:  94
재구매 고객 중 여성 수:  59
재구매 고객 중 남성 수:  59


In [35]:
#재구매 남성 인덱스 사용하여 36개를 랜덤 추출 
rep_g_idx=df.loc[(re_purchase)&(df['gender']==0)].index.tolist()
rep_g_idx=random.sample(rep_g_idx,35)

print(rep_g_idx)
print(len(rep_g_idx))

[900, 719, 714, 776, 323, 290, 825, 740, 874, 974, 803, 951, 540, 560, 735, 754, 539, 950, 651, 671, 683, 697, 227, 871, 863, 913, 562, 631, 635, 732, 931, 618, 310, 980, 750]
35


In [36]:
#인덱스를 활용하여 여성으로 변환

df.loc[rep_g_idx,'gender']=1

after_f=len(df[(re_purchase)&(df['gender']==1)])
after_m=len(df[(re_purchase)&(df['gender']==0)])
print('변환 후 재구매 여성 고객 수: ',after_f)
print('변환 후 재구매 남성 고객 수: ',after_m)

변환 후 재구매 여성 고객 수:  94
변환 후 재구매 남성 고객 수:  24


In [37]:
#재구매여부 변수를 데이터프레임에 추가

df['re_purchase']=re_purchase.replace([False,True],[0,1])
print(df['re_purchase'])

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: re_purchase, Length: 1000, dtype: int64


In [38]:
df.head()

,id,cusno,gender,age,region,product_20,product_21,price_20,price_21,quantity_20,quantity_21,amount_20,amount_21,re_purchase
0,1,2544,1,48,1,20,10,23225.68,96,91,2229665.28,2183282.01,0,0
1,2,3840,0,20,8,10,20,27716.36,92,5,2549905.12,52445.30,0,0
2,3,2859,0,54,7,10,10,24624.63,7,92,172372.41,2348875.00,1,0
3,4,4943,1,79,1,30,20,44141.70,51,20,2251226.70,204012.80,0,0
4,5,4927,0,10,3,30,20,26712.91,68,38,1816477.88,590478.20,0,0


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1000 non-null   int32  
 1   cusno        1000 non-null   int32  
 2   gender       1000 non-null   int32  
 3   age          1000 non-null   int32  
 4   region       1000 non-null   int32  
 5   product_20   1000 non-null   int32  
 6   product_21   1000 non-null   int32  
 7   price_20     1000 non-null   float64
 8   price_21     1000 non-null   int32  
 9   quantity_20  1000 non-null   int32  
 10  quantity_21  1000 non-null   float64
 11  amount_20    1000 non-null   float64
 12  amount_21    1000 non-null   int32  
 13  re_purchase  1000 non-null   int64  
dtypes: float64(3), int32(10), int64(1)
memory usage: 70.4 KB


##### 2. 20년도에 구매가 이루어졌으나, 21년도 구매 이력이 없는 이탈 고객 수의 80%가 남자
- 조건: 재구매 고객 제외

In [40]:
#재구매 고객이 아닌 고객 확인

n_all_re_purchase=len(df[(re_purchase==False)])
print('재구매 고객이 아닌 총 고객수: ',n_all_re_purchase)

재구매 고객이 아닌 총 고객수:  882


In [41]:
#현재 데이터 내 21년도 구매이력이 없는 고객 수 확인

before_m=df.loc[(re_purchase==False)&(df['gender']==0),
                ['product_21']].isnull().sum()
before_f=df.loc[(re_purchase==False)&(df['gender']==1),
                ['product_21']].isnull().sum()

print('이탈 남자 고객 수:\n',before_m)
print('이탈 여자 고객 수:\n',before_f)

이탈 남자 고객 수:
 product_21    0
dtype: int64
이탈 여자 고객 수:
 product_21    0
dtype: int64


In [42]:
#21년도 구매이력 없는 90명의 이탈고객 만들기

print('이탈 해야할 남자 고객 수: ',int(90*0.8),'명')
print('이탈 해야할 여자 고객 수: ',int(90*0.2),'명')

이탈 해야할 남자 고객 수:  72 명
이탈 해야할 여자 고객 수:  18 명


In [43]:
#90명의 이탈 고객: 남자:72명, 여자:12명의 인덱스 랜덤 추출
bye_m_idx=df.loc[(re_purchase==False)&(df['gender']==0)].index.tolist()
bye_m_idx=random.sample(bye_m_idx,72)

bye_f_idx=df.loc[(re_purchase==False)&(df['gender']==1)].index.tolist()
bye_f_idx=random.sample(bye_f_idx,18)

In [44]:
#추출된 인덱스로 21년도 구매이력 지우기 
df.loc[bye_m_idx,['product_21','price_21','quantity_21','amount_21']]=np.nan
df.loc[bye_f_idx,['product_21','price_21','quantity_21','amount_21']]=np.nan

after_m=df.loc[(re_purchase==False)&(df['gender']==0),
                ['product_21']].isnull().sum()
after_f=df.loc[(re_purchase==False)&(df['gender']==1),
                ['product_21']].isnull().sum()

print('이탈한 남자 고객 수:\n',after_m)
print('이탈한 여자 고객 수:\n',after_f)

이탈한 남자 고객 수:
 product_21    72
dtype: int64
이탈한 여자 고객 수:
 product_21    18
dtype: int64


In [45]:
#이탈 고객 여부 변수 생성

df['bye_purchase']=np.where(df['product_21'].isnull(),1,0)
df['bye_purchase'].value_counts()

0    910
1     90
Name: bye_purchase, dtype: int64

#### Q3. df 를 탐색적 분석을 통하여 인사이트를 도출하세요.

In [46]:
df.corr()

,id,cusno,gender,age,region,product_20,product_21,price_20,price_21,quantity_20,quantity_21,amount_20,amount_21,re_purchase,bye_purchase
id,1.000000,-0.033492,0.044722,0.010530,0.010237,-0.021646,0.038882,-0.032281,0.014808,-0.018543,-0.025738,0.002609,-0.003711,0.243007,-0.004721
cusno,-0.033492,1.000000,-0.031681,-0.018124,0.045879,0.050931,0.015965,-0.003011,0.021398,0.023929,0.001942,0.062306,0.055870,-0.043672,-0.012586
gender,0.044722,-0.031681,1.000000,-0.027116,-0.001476,0.039173,-0.027668,0.019647,0.009751,0.013351,0.026344,0.047681,0.042548,0.215520,-0.189951
age,0.010530,-0.018124,-0.027116,1.000000,-0.046480,0.007771,-0.057740,-0.028799,0.019117,0.008729,-0.029064,0.026291,-0.001762,-0.022512,-0.023282
region,0.010237,0.045879,-0.001476,-0.046480,1.000000,-0.065325,0.034207,-0.019255,-0.055529,0.008131,-0.050789,0.020932,0.024129,0.024482,0.058905
product_20,-0.021646,0.050931,0.039173,0.007771,-0.065325,1.000000,0.018110,0.018662,0.051633,0.000444,0.028182,-0.036980,-0.033714,-0.055099,0.005815
product_21,0.038882,0.015965,-0.027668,-0.057740,0.034207,0.018110,1.000000,-0.026793,0.024106,-0.031712,-0.004420,-0.007087,-0.002529,-0.009147,NaN
price_20,-0.032281,-0.003011,0.019647,-0.028799,-0.019255,0.018662,-0.026793,1.000000,-0.012461,-0.002285,0.651844,-0.054399,-0.395212,0.002984,0.008108
price_21,0.014808,0.021398,0.009751,0.019117,-0.055529,0.051633,0.024106,-0.012461,1.000000,0.011452,0.648802,-0.050820,-0.443529,0.014290,NaN
quantity_20,-0.018543,0.023929,0.013351,0.008729,0.008131,0.000444,-0.031712,-0.002285,0.011452,1.000000,0.002630,0.667178,0.362150,-0.029415,0.033992


In [47]:
df.corr().loc[:,['gender']].round(2)

,gender
id,0.04
cusno,-0.03
gender,1.00
age,-0.03
region,-0.00
product_20,0.04
product_21,-0.03
price_20,0.02
price_21,0.01
quantity_20,0.01


gender와 다른 변수들 간의 상관관계가 절대값 0.05 미만으로 연관성이 낮게 나타난 것과 달리,
re_purchase(재구매 여부)의 상관관계 계수는 0.22, bye_purchase(21년도 구매 이탈 여부)의 상관계수는 -0.19로 어느정도 상관관계를 유추할 수 있는 값이 나옴